In [2]:
#Bibliothèques
from __future__ import print_function
import pickle
from datetime import datetime
#from google_auth_oauthlib.flow import Flow, InstalledAppFlow
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from pydrive.drive import GoogleDrive
from pydrive.auth import GoogleAuth
import google.auth
import google
from googleapiclient.discovery import build
from google.cloud import storage
from google.oauth2 import service_account
from googleapiclient.errors import HttpError
import algo_sheet

import hachage
import json
import algo
import cv2
import numpy
import shutil,os,random

In [ ]:
#(3/3)
#(2/3)

#main de l'emetteur: saisie du message secret, copie des fichiers dans le stego dossier (1/3)


print(str(datetime.now()))
lis=os.listdir("C:\\Users\\Administrateur\\Downloads\\real_dataset")
#lis=os.listdir("C:\\Users\\Administrateur\\Desktop\\mes_images")
choix=random.choice(lis)
choix=("cat_0667.jpg")
print("Cover image extraite: ", choix)
print("\n")
positions=[]
rangement={}
algo.possibility()
algo.arrangement()
algo.algoapproach(algo.process("C:\\Users\\Administrateur\\Downloads\\real_dataset\\"+choix))

#algo.algoapproach(algo.process("C:\\Users\\Administrateur\\Desktop\\same_others\\pexels-sora-13916254.jpg"))
input_file=open('databloc.json','r')
var_json=json.load(input_file)

input_file2=open('correspondance.json','r')
var_json2=json.load(input_file2)

i=0
num=0
zero=""
secret=""
pos=0
tab=[]
find=False

client_secret='client_secrets.json'
api_name='drive'
api_version='v3'
SCOPES = ['https://www.googleapis.com/auth/drive']
service=algo_sheet.Create_Service(client_secret,api_name,api_version,SCOPES)

for elt in var_json.keys():
    rangement[elt]=len(var_json[elt])
with open('rangement.json', 'w') as monfichier:
    json.dump(rangement,monfichier)

#calcul de la clé pour les blocs(comprise entre 0 et 15 inclusivement)
key_block=0
avg=algo.average("C:\\Users\\Administrateur\\Downloads\\real_dataset\\"+choix)
#print(avg)
#input_file4=open('C:\\Users\\Administrateur\\Recepteur\\average.json','r')
input_file4=open('C:\\Users\\Administrateur\\average.json','r')
averages=json.load(input_file4)
#print(len(averages.keys()))
for elt in averages.keys():
    
    if elt==str(avg):
        break
    key_block+=1
print("La clé de chiffrement de bloc est: ", key_block)
rangm=key_block%10
#calcul de la clé pour les séquences(comprise entre 6 et 9 inclusivement)
if rangm>=6 and rangm<9:
    key_seq=rangm

elif rangm<3:
    key_seq=rangm+6
else:
    key_seq=6
#key_seq=rangm
print("La clé de chiffrement de séquence est: ", key_seq)
print("\n")
#chiffrement de l'ensemble des blocs utiles
algo.chiffrementbloc(rangement.keys(),key_block)
input_file3=open('chif.json','r')
chif_bloc=json.load(input_file3)


#association des répertoires faible et grande capacité
algo.association_bloc()

input_file4=open('association.json','r')
association=json.load(input_file4)
print("chif_bloc",chif_bloc)
print("association", association)
print("rangement",rangement)
print("\n")

message=input("Message secret à dissimuler: ")
print("\n")
message=(algo.bin_repr(message)).replace(" ","")
print("Message secret en binaire: ",message)#convertir le message en binaire
print("\n")

taille=len(message)#taille est une information qu'il faudra envoyer car c'est la taille du message meme, sans les zéros supplémentaires 
print("Taille du message: ", taille," bits")
print("\n")

if taille%15!=0:
    
    for i in range(15-taille%15):
        zero+="0"

secret=message+zero
#print(secret)
bloc=len(secret)
#print(bloc)

nbr=bloc//15
#print(nbr)
for i in range(nbr):
    tab.append(secret[15*i:15*i+15])
#print(tab)
for elt in tab:
    nbre=algo.structure(elt)
    for item in var_json.keys():
        if nbre==algo.structure(((var_json[item])[0])[1]):
            nbre=-1
            
            for val in var_json[item]:
                if elt==val[1]:
                    positions.append([item,val[0]])

print("Liste des positions cibles (numéro_bloc, numéro_séquence): ",positions)
print("\n")

#copier le fichier correspondant à la taille du message secret (il doit être le premier).
print("Copie en premier du fichier correspondant à la taille du message")
request=f"parents='{'1F907UMjHkIcbVbtgrls3chX5hyauo7s1'}' "
algo_sheet.search_file(taille,request)
print("\n")

print("Après chiffrement des numéros de séquence copie des fichiers correspondants dans le stégo dossier")
print("\n")

for elt in positions:
    #print(elt)
    num=0
    pos=0
    i=0
    j=0
    k=0
   # print("numéro de sequence en clair", elt[-1])
    #algo.chiffrement(algo.folder(elt) ,11,16)
    #quand la clé pour les séquences est égale à 9, s'il y'a utilisation du bloc associé alors on déchiffre en faisant l'inverse du chemin:chif-rech-mod
    #faire la recherhe dans le répertoire transitions jusqu'a arriver à ce répertoire. faire la recherche sur le fichier de numéro de séquence correspondant
    if int(algo.chiffrement(elt[-1],key_seq,10))>6435:
        #print("numéro de séquence chiffrée", algo.chiffrement(elt[-1],key_seq,10))
        seq=str((int(algo.chiffrement(elt[-1],key_seq,10)))%6435)
        #print("modulo du numéro de séquence chiffrée", seq)
        for ass in association.keys() :
            if elt [0]==ass:#si on est dans un bloc de grande capacité
                query=f"parents='{'1RnnfGxu6CnzNUjJYmQbBFJiGN9aHbczv'}' "
                response=service.files().list(q=query).execute()
                files=response.get('files')
                nextPageToken=response.get('nextPageToken')
                for file in files:
                    if i == algo.position(association[ass]):#on parcourt les répertoires jusqu'a trouver le répertoire de petite capacité associée
                       # print("numéro répertoire i" ,i)
                        #print(file['id'])
                        query=f"parents='{file['id']}' "
                        #response=service.files().list(q=query).execute()
                        #fichiers=response.get('files')
                        #nextPageToken=response.get('nextPageToken')
                        algo_sheet.search_file(algo.position_seq(seq),query)#c'est dans le répertoire de petite capacité associée qu'on recherche le fichier
                    i+=1
    else:
        seq=algo.chiffrement(elt[-1],key_seq,10) # on entre dans ce cas ci ssi keyseq=9
        if int(seq)>5324:
            for ass in association.keys() :
                if elt [0]==ass:
                    query=f"parents='{'1RnnfGxu6CnzNUjJYmQbBFJiGN9aHbczv'}' "
                    response=service.files().list(q=query).execute()
                    files=response.get('files')
                    nextPageToken=response.get('nextPageToken')
                    for file in files:
                        if k == algo.position(association[ass]):
                            #print("numéro répertoire k" ,k)
                            query=f"parents='{file['id']}' "
                        #response=service.files().list(q=query).execute()
                        #fichiers=response.get('files')
                        #nextPageToken=response.get('nextPageToken')
                            algo_sheet.search_file((algo.position_seq(seq))%6435,query)
                        k+=1
            
        else:
            #print(seq)
            query=f"parents='{'1RnnfGxu6CnzNUjJYmQbBFJiGN9aHbczv'}' "
            response=service.files().list(q=query).execute()
            files=response.get('files')
            nextPageToken=response.get('nextPageToken')
            for file in files:
                #print(elt[0])
                if j == algo.position(elt[0]):
                    #print("numéro répertoire j" ,j)
                    query=f"parents='{file['id']}' "
                        #response=service.files().list(q=query).execute()
                        #fichiers=response.get('files')
                        #nextPageToken=response.get('nextPageToken')
                    if elt[0] in association.keys():
                        algo_sheet.search_file(algo.position_seq(seq),query)
                        
                    
                    else:
                        algo_sheet.search_file(5000+algo.position_seq(seq),query)
                j+=1
print(str(datetime.now()))

In [ ]:
#main récepteur: parcours le stego-dossier et renvoie les séquences pour chacun des fichiers 
print(str(datetime.now()))
#message="00100011011000110110010101100011011010010010000001100101011100110111010000100000011101010110111000100000011011010110010101110011011100110110000101100111011001010010000001110100011100101110100001110011001000000111001101100101011000110111001001100101011101000010111000100000010001010111100011101001011000110111010101110100011001010111101000100000011101000111001011101000011100110010000001110010011000010111000001101001011001000110010101101101011001010110111001110100001000000111010001101111011101010111010001100101011100110010000001101100011001010111001100100000011001000110100101110010011001010110001101110100011010010111011001100101011100110011101000100000011010000110010101110101011100100110010100100000011001000010011101100101011011010110001001100001011100100111000101110101011001010110110101100101011011100111010000100000001100010011001101101000001011000010000001101100011010010110010101110101001110100110000111101001011100100110111101110000011011110111001001110100001000000110001101100101011011100111010001110010011000010110110"
message="0010001101100011011001010110001101101001001000000110010101110011011101000010000001110101011011100010000001101101011001010111001101110011011000010110011101100101001000000111010001110010111010000111001100100000011100110110010101100011011100100110010101110100001011100010000001000101011110001110100101100011011101010111010001100101011110100010000001110100011100101110100001110011001000000111001001100001011100000110100101100100011001010110110101100101011011100111010000100000011101000110111101110101011101000110010101110011001000000110110001100101011100110010000001100100011010010111001001100101011000110111010001101001011101100110010101110011001110100010000001101000011001010111010101110010011001010010000001100100001001110110010101101101011000100110000101110010011100010111010101100101011011010110010101101110011101000010000000110001001100110110100000101100001000000110110001101001011001010111010100111010011000011110100101110010011011110111000001101111011100100111010000100000011000110110010101101110011101000111001001100001011011000110010100100000011101000111001011101000011100110010000001110011011001010110001101110010011001010111010000101110010001010111100011101001011000110111010101110100011001010111101000100000011101000110111101110101011101000110010101110011001000000110110001100101011100110010000001100100011010010111001001100101011000110111010001101001011101100110010101110011"
taille=len(message)
client_secret='C:\\Users\\Administrateur\\Recepteur\\client_secrets.json'
api_name='drive'
api_version='v3'
SCOPES = ['https://www.googleapis.com/auth/drive']
service=algo_sheet.Create_Service(client_secret,api_name,api_version,SCOPES)
liste=[]
rangement={}
category=[]
add_blocks=[]#les blocs à rajouter pour rendre la stego image bruitée adéquate quand elle ne l'est pas
mes=[]
key_block=12
blocks=[]
image="C:\\Users\\Administrateur\\state_art1\\sp_images\\cat_0667.jpg"#prendre une image bruitée
algo.possibility()
algo.arrangement()
algo.algoapproach(algo.process(image))
input_file=open('dataArrangement.json', 'r')
var_jsonA=json.load(input_file)

    
#algo.algoapproach(algo.process("C:\\Users\\Administrateur\\Desktop\\same_others\\pexels-sora-13916254.jpg"))
input_file=open('databloc.json','r')
var_json=json.load(input_file)
print(len(var_json.keys()))
if len(var_json.keys())<16:
    for elt in var_json.keys():
        category.append(algo.structure(((var_json[elt])[0])[-1]))
   
    for elt in range(16):
        if elt not in category:
            add_blocks.append(elt)
      
    for block in add_blocks:
        for elt in var_jsonA.keys():
            
            
            if str(block)+"0"==elt:
                print(str(block)+"0")
                blocks.append(var_jsonA[elt])
                break
    k=0
    
    for b in blocks:
        k=0
        
        for elt in var_json.copy():

            if k==len(var_json.keys()) -1:
                var_json[str(int(elt)+1)]=b
                
            k+=1
    print(len(var_json.keys()))
    with open("databloc.json","w") as monfich:
        json.dump(var_json,monfich)
for elt in var_json.keys():
    rangement[elt]=len(var_json[elt])
print("rangement",rangement)
with open('rangement.json', 'w') as monfichier:
    json.dump(rangement,monfichier)

algo.chiffrementbloc(rangement.keys(),key_block)
input_file1=open('chif.json','r')
var_json1=json.load(input_file1)

    
print("chiffrement_bloc",var_json1)
algo.association_bloc()
input_file2=open('association.json','r')
var_json2=json.load(input_file2)

print("association",var_json2)
#pour les stego-images bruitées qui ne sont plus adéquates, il faudra rajouter les blocs utiles manquants dans le fichier 'association'

key_seq=8

query=f"parents='{'1_kGPAA1ZyOkglYoEPLzLLCUB7nBWJjW1'}' " 
response=service.files().list(q=query).execute()
files=[]
nextPage=response.get('files')
nextPageToken=response.get('nextPageToken',None)
while True:
    files=nextPage
    for file in files:
        print("nom du fichier recherché", file['name'])
        liste.append(algo_sheet.search_position(file['name']))
    print("liste des couples", liste)
    if nextPageToken is None:
        break
    response=service.files().list(q=query,pageToken=nextPageToken).execute()
    nextPage=response.get('files')
    nextPageToken=response.get('nextPageToken',None)
with open ("couples.json","w") as couple:
    json.dump(liste,couple)
for elt in liste[:-1]: #on ne recherche pas le dernier fichier car il s'agit du fichier qui code la taille du msg secret et non une séquence
    print("recherche fichier", elt)
    block=algo.reversePosition(elt[0])#calculer la séquence de hachage de l'image (databloc), créer le fichier rangement, 
    #créer le fichier chif,
    #print("block", block)
    
    if algo.capacity(block) >= 1365:
        #print("algo.capacity(block)",algo.capacity(block))
        mes.append(algo.deciphered([block,algo.decryption_seq(algo.reversePosition_seq(elt[1]),key_seq)]))
        #print(mes)
    else:
        if elt[1]>=5000:
            #print("elt[1]",elt[1])
            mes.append(algo.deciphered([block,algo.decryption_seq(algo.reversePosition_seq(elt[1]-5000),key_seq)]))
        else:
           
            mes.append(algo.deciphered([algo.reverse_asso(block),algo.decryption_seq(int(algo.reversePosition_seq(elt[1]))+6435,key_seq)]))
print(mes)
j=0
i=0
for segment in mes:
    if segment==None:
        i+=1
        mes[j]="               "
    j+=1
print(i/len(mes))
elts=liste[-1]
mes[0]=(mes[0])[:(elts[-1]%15)]

mes=''.join(list(reversed(mes)))

monfichier.close()
input_file1.close()
input_file.close() 
input_file2.close()

print(algo.ber(message,mes)/(taille-i*15))
str(datetime.now())

In [ ]:
#Décodage et affichage du message final-possible si absence de none

input_string=int(mes, 2);

#Obtain the total number of bytes
Total_bytes= (input_string.bit_length()+7) // 8

input_array = input_string.to_bytes(Total_bytes, "big")

#Convert the bytes to an ASCII value and display it on the output screen
ASCII_value=input_array.decode('iso-8859-1')
print(ASCII_value)

In [ ]:
#Algorithme de récupération-but:réduction des none par réattribution des catégories de bloc
print(str(datetime.now()))
import algo
import json 
key_seq=8

def reversePosition(pos):
    chif_bloc={'5': 0, '8': 1, '10': 2, '31': 3, '74': 4, '161': 5, '242': 6, '287': 7, '597': 8, '884': 9, '1438': 10, '1441': 11, '0': 12, '1': 13, '2': 14, '3': 15}

    for elt in chif_bloc.keys() :
        if chif_bloc[elt]==pos :
            return elt

    #if elt[0]==num:
    
def reverse_asso(block): 
    association ={'5': '161', '8': '242', '10': '597', '31': '884', '74': '1438', '287': '1441', '1': '0', '2': '3'} 

    for ass in association.keys():
        if association[ass]==block:
            return ass  
def capacity(numbloc): 
    rangement={'0': 105, '1': 5005, '2': 1365, '3': 455, '5': 6435, '8': 3003, '10': 6435, '31': 3003, '74': 5005, '161': 455, '242': 15, '287': 1365, '597': 1, '884': 105, '1438': 15, '1441': 1}

    for elt in rangement.keys() :
        if str(elt)==str(numbloc) :
            return rangement[elt]
chif_bloc={'5': 0, '8': 1, '10': 2, '31': 3, '74': 4, '161': 5, '242': 6, '287': 7, '597': 8, '884': 9, '1438': 10, '1441': 11, '0': 12, '1': 13, '2': 14, '3': 15}
categories={1:[0,15], 15:[1,14], 105:[2,13], 455:[3,12], 1365:[4,11], 3003:[5,10], 5005:[6,9], 6435:[7,8]} 
temp={}
blocks=[]
message="001000110110001101100101011000110110100100100000011001010111001101110100001000000111010101101110001000000110110101100101011100110111001101100001011001110110010100100000011101000111001011101000011100110010000001110011011001010110001101110010011001010111010000101110001000000100010101111000111010010110001101110101011101000110010101111010001000000111010001110010111010000111001100100000011100100110000101110000011010010110010001100101011011010110010101101110011101000010000001110100011011110111010101110100011001010111001100100000011011000110010101110011001000000110010001101001011100100110010101100011011101000110100101110110011001010111001100111010001000000110100001100101011101010111001001100101001000000110010000100111011001010110110101100010011000010111001001110001011101010110010101101101011001010110111001110100001000000011000100110011011010000010110000100000011011000110100101100101011101010011101001100001111010010111001001101111011100000110111101110010011101000010000001100011011001010110111001110100011100100110000101101100"
#rangement={}
key_block=12
taille=len(message)
messages=['001100000001010', '100011100100110', '010110111001110', '110001001100010', None, '110111101110010', '101111011100000', '010010111001001', '101001100001111', '101011101010011', None, '000110000110000', None, '011000100110011', '010000011010001', '001010110111001', '010101101101011', '001011101010110', '010111001001110', '010010000110111', '011001010110110', '110010000100111', '001101000000110', '101010111001001', '100001100101011', '001000000110011', '010111001100111', '101110110011001', '011101000110100', '110010101100011', '101001011100100', '000000111000011', '010101110011001', None, '010111001100100', '101110100011001', '011011110111010', '110000100100000', '101110011101000', '011010110010101', None, None, '100110000101110', None, '111010000111001', '111010001110010', None, '101000110010101', '001101110101011', '000111010010110', '000100010101111', '001100000010100', '011001010111010', '110001101110010', '110011011001010', None, '110010100001001', '000011101000111', '110110010100100', '101100001011001', '011100110111001', '110110101100101', None, None, None, '000011001010111', '110110100100100', '101100101011000', '001000110110001']
listes=[[7, 530], [0, 2245], [12, 3838], [4, 3991], [11, 3750], [1, 1490], [2, 3169], [0, 1354], [2, 2338], [13, 2053], [5, 678], [7, 310], [5, 2106], [0, 2085], [9, 4147], [2, 543], [13, 851], [2, 653], [8, 3743], [5, 3685], [2, 1414], [0, 5055], [9, 4003], [2, 3630], [0, 2935], [3, 791], [12, 3869], [13, 2558], [0, 2670], [2, 4621], [0, 4103], [3, 1032], [8, 4543], [5, 232], [0, 1846], [2, 4044], [1, 513], [11, 4127], [2, 4034], [2, 1623], [4, 4623], [5, 214], [0, 3679], [7, 1685], [2, 4607], [2, 4529], [7, 1286], [0, 4007], [13, 323], [0, 687], [0, 271], [7, 534], [2, 1416], [2, 3380], [2, 4735], [7, 1720], [4, 4014], [0, 202], [0, 4453], [0, 3334], [12, 4436], [13, 2312], [7, 2094], [5, 55], [4, 4209], [0, 142], [0, 4476], [0, 3418], [10, 3809], [0, 1032]]
i,j,l,m,n=0,1,0,0,0 
list_cases=[] 
capacities=[1,15,105,455,1365,3003,5005,6435] 
couples_prob=[]
mes=[]
concerned_block=[]
deuxieme=[]

for segment in messages: 
    if segment==None: 
        couples_prob.append(listes[i]) 
    i+=1 


for c in couples_prob: 
    if c[0] not in concerned_block: 
        concerned_block.append(c[0])
    
for elt in couples_prob:

#print("elt ",elt)
    couples=[]
    for couple in couples_prob:
        if couple[0]==elt[0]:
            couples.append(couple)
       
    couples_prob=[ele for ele in couples_prob if ele not in couples]
    #print(couples)

    for couple in couples:
        block=reversePosition(couple[0])
        if block not in temp:
            temp[block]=[0]
        if capacity(block) >= 1365:
            
            seq=algo.decryption_seq(algo.reversePosition_seq(couple[1]),key_seq)
            
            if int(seq)>capacity(block) and int(seq)<6435: 
                for ele in (temp[block])[1:]:
                    if (ele[0])[-1]<int(seq) and (ele[0])[-1]>=1365:
                        temp[block].remove(ele)
                for c in chif_bloc.keys():
                    
                    if capacity(c)>int(seq):
                        #print(couple,block,c,seq,capacity(c))
                        p=capacity(block)
                        if [[block,capacity(c)],[c,p]] not in temp[block]:
                            temp[block].append([[block,capacity(c)],[c,p]])
                        
                seq=algo.decryption_seq(int(algo.reversePosition_seq(couple[1]))+6435,key_seq)
                if seq[0]!='0' and ((temp[block])[0])!=-1 and int(seq)<6435:
                    for c in chif_bloc.keys():
                        if capacity(c) <=455:
                            p=capacity(block)
                            if [[block,capacity(c)],[c,p]] not in temp[block]:
                                temp[block].append([[block,capacity(c)],[c,p]])
                            
                else:
                    ((temp[block])[0])=-1
                #print(((temp[block])[0])[-1])
                            
            
            else: #on avait un bloc de faible capacité 
                for c in chif_bloc.keys():
                    if capacity(c) <=455 and ((temp[block])[0])!=-1:
                        p=capacity(block)
                        if [[block,capacity(c)],[c,p]] not in temp[block]:
                            temp[block].append([[block,capacity(c)],[c,p]])
                        

        else:
            
            if couple[1]>=5000 and couple[1]<6111 :#on avait un bloc de faible capacité mais de capacité supérieure à celle actuelle
                seq=algo.decryption_seq(algo.reversePosition_seq(couple[1]-5000),key_seq)
                
                for c in chif_bloc.keys():
                    if capacity(c)>int(seq) and cap<=455 and ((temp[block])[0])!=-1 :
                        perm=capacity(block)
                        if [[block,capacity(c)],[c,perm]] not in temp[block]:
                            temp[block].append([[block,capacity(c)],[c,perm]])
                       
            else: #le bloc associé etait d'une capacité supérieure à celle actuelle
    
                seq=algo.decryption_seq(int(algo.reversePosition_seq(couple[1]))+6435,key_seq)
                if int(seq)>=6435 or seq[0]=="0":
                    seq=algo.decryption_seq(algo.reversePosition_seq(couple[1]),key_seq)
            
                    if block not in temp:
                        temp[block]=[0]
                    for ele in (temp[block])[1:]:
                        if (ele[0])[-1]<int(seq) and (ele[0])[-1]>=1365:
                            temp[block].remove(ele)
                    for c in chif_bloc.keys():
                        if capacity(c)>int(seq):
                            p=capacity(block)
                            if [[block,capacity(c)],[c,p]] not in temp[block]:
                                temp[block].append([[block,capacity(c)],[c,p]])
                else:
                    
                     deuxieme.append(couple)
temporary=[]
for ele in temp:
    if ele not in temporary:
        temporary.append(ele)

print("\n")
for ele in temp:
    print(ele,temp[ele])
    if len((temp[ele])[1:])>=1:
        blocks.append((temp[ele])[1:])
print("\n")

print(blocks)
#for ele in temporary: #on doit conserver pour chaque bloc la capacité max
print("\n")
#print(deuxieme)
print(len(blocks))
cas=[]
while j<len(blocks):
    cas=[]
    
    if j==1:
        for e in blocks[0]:
            for el in blocks[j]:
                if (e[1])[0]!=(el[1])[0]:
                    cas.append([e,el])
        #print(liste,"\n")
    if j==2:
        for e in blocks[0]:
            for el in blocks[j]:
                if ((e[0])[1])[0]!=(el[1])[0] and ((e[1])[1])[0]!=(el[1])[0]:
                    cas.append([e[0],e[1],el])
        #print(liste,"\n")
    if j>=3:
        print("oui")
        for e in blocks[0]:
            for el in blocks[j]:
                test=0
                liste_perm=[]
                for elt in e:
                    if (elt[1])[0]==(el[1])[0] or (elt[0])[0]==(el[1])[0] :
                        test=1
                        break
                if test==0:
                    for elt in e:
                        liste_perm.append(elt)
                    
                    liste_perm.append(el)
                    cas.append(liste_perm)
        #print(liste,"\n")
                    
   
    blocks[0]=cas
    
  
    j+=1
if len(cas)>0:
    print(len(cas))
    print(deuxieme)
else:
    for elt in blocks:
        for ele in elt:
            cas.append(ele)
    print(len(cas))
    
#print(liste)
print(str(datetime.now()))      

In [ ]:
#calcul du nombre de none et du ber pour chaque cas de figure (nouvelle réattribution de categories de bloc)
print(str(datetime.now()))
rangement={}
list_mes,list_bers,list_none=[],[],[]
input_file=open('databloc.json','r')
var_json=json.load(input_file) 

for eltb in cas:
    
    #input_file=open('C:\\Users\\Administrateur\\approach\\databloc_speckle\\dbpexels-kristina-paukshtite-10520481 (1).jpg.json','r')
    #var_json=json.load(input_file) 
    var_json1={}
    mes=[]
    couples_prob=[]

    var_json1=var_json
    for bloc in eltb:
        perm=var_json1[(bloc[0])[0]]
        var_json1[(bloc[0])[0]]=var_json1[(bloc[1])[0]]
        var_json1[(bloc[1])[0]]=perm
    for elt in var_json1.keys():
        rangement[elt]=len(var_json1[elt])

    with open('rangement.json', 'w') as monfichier:
        json.dump(rangement,monfichier)
    with open('databloc.json', 'w') as monfichier:
        json.dump(var_json1,monfichier)
    algo.chiffrementbloc(rangement.keys(),key_block)
    input_file3=open('chif.json','r')
    chif_bloc=json.load(input_file3)

    algo.association_bloc()
    input_file4=open('association.json','r')
    association=json.load(input_file4)
    #print(rangement," ",chif_bloc," ",association,"\n")
    for elt in listes[:-1]: 
        block=algo.reversePosition(elt[0])
        if algo.capacity(block) >= 1365:
            mes.append(algo.deciphered([block,algo.decryption_seq(algo.reversePosition_seq(elt[1]),key_seq)]))
        else:
            if elt[1]>=5000:
                mes.append(algo.deciphered([block,algo.decryption_seq(algo.reversePosition_seq(elt[1]-5000),key_seq)]))
            else:
                mes.append(algo.deciphered([algo.reverse_asso(block),algo.decryption_seq(int(algo.reversePosition_seq(elt[1]))+6435,key_seq)]))
        if mes[-1]==None:
            couples_prob.append(elt)
            
    l=0
    m=0
    for segment in mes:
        if segment==None:
            l+=1
            mes[m]="               "
        m+=1
    #print(l/len(mes))
    elts=listes[-1]

    mes[0]=(mes[0])[:(elts[-1]%15)]

    mes=''.join(list(reversed(mes)))
    list_mes.append(mes)
    list_none.append(l)
    bers=algo.ber(message,mes)/(taille-l*15)
    print(l,bers)
    list_bers.append(bers)
    
    if l<=5:
        
        break
        
minimn=min(list_none)
print(minimn)
print(min(list_bers))
index=0
for elt in list_none:
    if elt==minimn:
        
        break
    index+=1
minimb=list_bers[index]
print(index)
print(str(datetime.now()))


In [ ]:
#list_mes[0]="010111001001110001011101010110010101101101011001010110111001110100001000000011000100110011110000000011001000100000110111001000000101101101011101010011101001100001111010010111001001101111011100000110111101110010010010001010011001100011011001010110111001110100011100100110000101100010"
input_string=int("001000110110001101100100011000110110100100100000001001010011001101110100001000000111010101101110001000000110110101100101001100110111001101100001011001110110010100100000001101000111001011101000011100110010000001110011011001010110001101110010011001010111010000101110001000000100010101111000111010010111001101110101011101000110010101111010001000000011010001110010111000000111001100100000011100100110000101110000011010010110010001100101011011010110010101100110011101000010000001110100010001110111010101110100011001010111001100100000011011000110010101110011001000000110010001101001011100100110010101100011011101000110100101110110011001010111001100111010001000000110100001100101011101010111001001100101001000000110010000100111011001010110110101100010011000010111001001110001011101010110010101101101011001010010111001110100001000000011000100110011011010000010110000100000011011000110100001100101011101100011101001100001111010010111001001101111011110000110111101110010011101000010000001100011011001010110111001110100011100100110000101101100",2)
#print(len("110011011001010110001101110010011001010111010110010111111110000100010101111000111010010110001101110101011101000110010101"))
#Obtain the total number of bytes
Total_bytes= (input_string.bit_length()+7) // 8

input_array = input_string.to_bytes(Total_bytes, "big")

#Convert the bytes to an ASCII value and display it on the output screen
ASCII_value=input_array.decode('iso-8859-1')
print(ASCII_value)
   

In [2]:
#calcul des séquences de hachage des images bruitées

import os
import algo
lis=os.listdir("C:\\Users\\Administrateur\\state_art1\\speckleNoise_images")
for i in range(len(lis)):
    image="C:\\Users\\Administrateur\\state_art1\\speckleNoise_images\\"+lis[i]
    #chemin="C:\\Users\\Administrateur\\approach\\databloc_gn\\"+lis[i]
    #algo.contrast(algo.process(image)).save(""+chemin)
    algo.algoapproach(algo.process(image),lis[i])

In [ ]:
#traitement des couples non traités
for elt in deuxieme:
    if capacity(association(reversePosition(elt[0])))<elt[-1]:
        for c in chif_bloc:
            if capacity(c)>elt[-1]:
                if c not in temp:
                    temp[block]=[]
                else:
                    for e in temp[block]:
                        if e<elt[-1]:
                            temp[block].remove(e)
                    temp[block].append([[block,capacity(c)],[c,p]])

In [ ]:
#calcul du nombre de none et du ber pour chaque cas de figure (nouvelle réattribution de categories de bloc)
print(str(datetime.now()))
list_mes,list_bers,list_none=[],[],[]
for eltb in cas:
    input_file=open('C:\\Users\\Administrateur\\approach\\databloc_sp\\dbcat_0480.jpg.json','r')
    var_json=json.load(input_file) 
    var_json1={}
    mes=[]

    var_json1=var_json
    
    perm=var_json1[(eltb[0])[0]]
    var_json1[(eltb[0])[0]]=var_json1[(eltb[1])[0]]
    var_json1[(eltb[1])[0]]=perm
    for elt in var_json1.keys():
        rangement[elt]=len(var_json1[elt])

    with open('rangement.json', 'w') as monfichier:
        json.dump(rangement,monfichier)
    with open('databloc.json', 'w') as monfichier:
        json.dump(var_json1,monfichier)
    algo.chiffrementbloc(rangement.keys(),key_block)
    input_file3=open('chif.json','r')
    chif_bloc=json.load(input_file3)

    algo.association_bloc()
    input_file4=open('association.json','r')
    association=json.load(input_file4)
    for elt in listes[:-1]: 
        block=algo.reversePosition(elt[0])
        if algo.capacity(block) >= 1365:
            mes.append(algo.deciphered([block,algo.decryption_seq(algo.reversePosition_seq(elt[1]),key_seq)]))
        else:
            if elt[1]>=5000:
                mes.append(algo.deciphered([block,algo.decryption_seq(algo.reversePosition_seq(elt[1]-5000),key_seq)]))
            else:
                mes.append(algo.deciphered([algo.reverse_asso(block),algo.decryption_seq(int(algo.reversePosition_seq(elt[1]))+6435,key_seq)]))
            
    l=0
    m=0
    for segment in mes:
        if segment==None:
            l+=1
            mes[m]="               "
        m+=1
    #print(l/len(mes))
    elts=listes[-1]

    mes[0]=(mes[0])[:(elts[-1]%15)]

    mes=''.join(list(reversed(mes)))
    list_mes.append(mes)
    list_none.append(l)
    bers=algo.ber(message,mes)/(taille-l*15)
    print(l,bers)
    list_bers.append(bers)
#minimb=min(list_bers)
minimn=min(list_none)
print(minimn)
print(min(list_bers))
index=0
for elt in list_none:
    if elt==minimn:
        
        break
    index+=1
minimb=list_bers[index]
print(index)
print(str(datetime.now()))


In [ ]:
def calcul(chaine):
    
    input_string=int(chaine, 2);

    Total_bytes= (input_string.bit_length()+7) // 8

    input_array = input_string.to_bytes(Total_bytes, "big")

    ASCII_value=input_array.decode('iso-8859-1')
    return ASCII_value



def extraction(msg_bin):
    rang_none=0
    
    compt=0
    suite=""
    for i in range(len(msg_bin)):
        if msg_bin[i]==" " :
            if msg_bin[i-1]!=" ":

                compt=rang_none%8
                r=8-compt
                print(rang_none,r,compt)

                if r%8<7:
                    print(rang_none,r,compt,i)
                    if len(msg_bin[rang_none+r%8:i])%8==0:
                        chaine=msg_bin[rang_none+r%8:i]
                        print("mult",chaine)
                    else:
                        s=len(msg_bin[rang_none+r%8:i])%8
                        chaine=msg_bin[rang_none+r%8:i-s]
                        print(s,len(msg_bin[rang_none+r%8:i-s]),chaine)
                   
                else:
                    chaine="0"+msg_bin[rang_none+r:i]
                print(suite)
                suite+=" "+calcul(chaine)
                print(suite)
            rang_none=i
    if rang_none!=i:
        compt=rang_none%8
        r=8-compt
        if r<7:
            if len(msg_bin[rang_none+r%8:i])%8==0:
                chaine=msg_bin[rang_none+r%8:i]
            else:
                s=len(msg_bin[rang_none+r%8:i])%8
                chaine=msg_bin[rang_none+r%8:i-s]
        else:
            chaine="0"+msg_bin[rang_none+r:i]
            print(suite)
            suite+=" "+calcul(chaine)
            print(suite)
        suite+=" "+calcul(chaine)
        
    return suite
                    

msg_bin="001000110110001101100101011000110110100100100000011001010111001101110100001000000111010101101110001000000                              101100001011001110110010100100000011101000111001011101000011100110010000001110011011001010110001101110010011001010111010000101110001000000100010101111000111010010110101011111101011101000110010101111010001000000111010001110010111010000111001100100000011100100110000101110000011010010110010001100101011011010110010101101110011101000010000001110100011011110111010101110100011001010111001100100000011011000110010101110011001000000110010001101001011100100110010101100011011101000110100               111101011110110010001000000110100001100101011101010111001001100101001000000110010000100111011001010110110101100010011000010111001001110001011101010110111011001111101001010110111001110100001000000011000100110011011010000010110000100000011011000110100101100               101001100001111010010111001001101111011100000110111101110010011101000010000001100011011001111100111110110100011100100110000101101100"
print(extraction(msg_bin))

In [ ]:
#création des 17 répertoires(16 rep utiles et un stego rep)

import algo_sheet
liste=[]
client_secret='client_secrets.json'
api_name='drive'
api_version='v3'
SCOPES = ['https://www.googleapis.com/auth/drive']
service=algo_sheet.Create_Service(client_secret,api_name,api_version,SCOPES)
with open ('C:\\Users\\Administrateur\\Desktop\\names_folder.txt','r') as fich:
    for line in fich:
        liste.append(line.rstrip('\n'))
for elt in range(16):
    file_metadata={'name':liste[elt], 'mimeType':'application/vnd.google-apps.folder','parents':['1RnnfGxu6CnzNUjJYmQbBFJiGN9aHbczv']}
    service.files().create(body=file_metadata).execute()

In [ ]:
#création des fichiers de couverture dans le cloud

from pydrive.drive import GoogleDrive
from pydrive.auth import GoogleAuth
import os
import algo_sheet
gauth = GoogleAuth()
     
drive = GoogleDrive(gauth)
names=[]
with open ('C:\\Users\\Administrateur\\Desktop\\list_names14.txt','r') as fich:
    for line in fich:
        names.append(line.rstrip('\n'))

for j in range(6000,6435,1):
    f = drive.CreateFile({'title': names[j]+".txt", 'parents': [{'id': '1YmakEUHG_SJrJXsztjJ7hxRl5pw2EBvW'}] })
    f.SetContentString("new client")
    f.Upload()